In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import re
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional

In [ ]:
human_dna = pd.read_table('human_data.txt')
human_dna.head()

chimp_dna = pd.read_table('chimp_data.txt')
chimp_dna.head()

dog_dna = pd.read_table('dog_data.txt')
dog_dna.head()

,sequence,class
0,ATGCCACAGCTAGATACATCCACCTGATTTATTATAATCTTTTCAA...,4
1,ATGAACGAAAATCTATTCGCTTCTTTCGCTGCCCCCTCAATAATAG...,4
2,ATGGAAACACCCTTCTACGGCGATGAGGCGCTGAGCGGCCTGGGCG...,6
3,ATGTGCACTAAAATGGAACAGCCCTTCTACCACGACGACTCATACG...,6
4,ATGAGCCGGCAGCTAAACAGAAGCCAGAACTGCTCCTTCAGTGACG...,0


In [ ]:
#Let's define a function to collect all possible overlapping k-mers of a specified length from any sequence string.
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def Kmers_funct(seq, size=6):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

In [ ]:
human_dna['words'] = human_dna.apply(lambda x: Kmers_funct(x['sequence']), axis=1)
human_dna = human_dna.drop('sequence', axis=1)
chimp_dna['words'] = chimp_dna.apply(lambda x: Kmers_funct(x['sequence']), axis=1)
chimp_dna = chimp_dna.drop('sequence', axis=1)
dog_dna['words'] = dog_dna.apply(lambda x: Kmers_funct(x['sequence']), axis=1)
dog_dna = dog_dna.drop('sequence', axis=1)

In [ ]:
#Since we are going to use scikit-learn natural language processing tools to do the k-mer counting, we need to now convert the lists of k-mers for each gene into #string sentences of words that the count vectorizer can use. We can also make a y variable to hold the class labels.
human_texts = list(human_dna['words'])
for item in range(len(human_texts)):
    human_texts[item] = ' '.join(human_texts[item])
#separate labels
y_human = human_dna.iloc[:, 0].values # y_human for human_dna

In [ ]:
#Now let's do the same for chimp and dog.
chimp_texts = list(chimp_dna['words'])
for item in range(len(chimp_texts)):
    chimp_texts[item] = ' '.join(chimp_texts[item])

In [ ]:
chimp_texts[0]

'atgccc tgcccc gcccca ccccaa cccaac ccaact caacta aactaa actaaa ctaaat taaata aaatac aatacc ataccg taccgc accgcc ccgccg cgccgt gccgta ccgtat cgtatg gtatga tatgac atgacc tgaccc gaccca acccac cccacc ccacca caccat accata ccataa cataat ataatt taatta aattac attacc ttaccc tacccc accccc ccccca ccccat cccata ccatac catact atactc tactcc actcct ctcctg tcctga cctgac ctgaca tgacac gacact acacta cactat actatt ctattt tatttc atttct tttctc ttctcg tctcgt ctcgtc tcgtca cgtcac gtcacc tcaccc caccca acccaa cccaac ccaact caacta aactaa actaaa ctaaaa taaaaa aaaaat aaaata aaatat aatatt atatta tattaa attaaa ttaaat taaatt aaattc aattca attcaa ttcaaa tcaaat caaatt aaatta aattac attacc ttacca taccat accatc ccatct catcta atctac tctacc ctaccc tacccc accccc cccccc ccccct cccctc ccctca cctcac ctcacc tcacca caccaa accaaa ccaaaa caaaac aaaacc aaaccc aaccca acccat cccata ccataa cataaa ataaaa taaaaa aaaaat aaaata aaataa aataaa ataaaa taaaaa aaaaaa aaaaac aaaact aaacta aactac actaca ctacaa tacaat acaata caataa aataaa ataaa

In [ ]:
#separate labels
y_chim = chimp_dna.iloc[:, 0].values # y_chim for chimp_dna


In [ ]:
dog_texts = list(dog_dna['words'])
for item in range(len(dog_texts)):
    dog_texts[item] = ' '.join(dog_texts[item])
#separate labels
y_dog = dog_dna.iloc[:, 0].values  # y_dog for dog_dna

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(human_texts)
encoded_docs = tokenizer.texts_to_sequences(human_texts)
max_length = max([len(s.split()) for s in human_texts])
X = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

X_train,X_test,y_train,y_test = train_test_split(X,y_human,
                                                 test_size=0.20,random_state=42)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
Y_train = to_categorical(y_train)

In [ ]:
Y_test = to_categorical(y_test)

In [ ]:
# Build neural network architecture

adam = Adam(learning_rate=0.005)

model = Sequential()
model.add(Embedding(vocab_size, 10))
model.add((LSTM(20)))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 10)          44700     
                                                                 
 bidirectional (Bidirectiona  (None, 40)               4960      
 l)                                                              
                                                                 
 dense (Dense)               (None, 7)                 287       
                                                                 
Total params: 49,947
Trainable params: 49,947
Non-trainable params: 0
_________________________________________________________________


In [ ]:

history = model.fit(X_train,
                    Y_train, 
                    validation_data=(X_test, Y_test),
                    verbose=1,
                    batch_size=200,
                    epochs=5,
                     
                    )

Epoch 1/5
18/18 [==============================] - 1273s 70s/step - loss: 1.8413 - accuracy: 0.2897 - val_loss: 1.7788 - val_accuracy: 0.3025
Epoch 2/5
18/18 [==============================] - 1215s 68s/step - loss: 1.6189 - accuracy: 0.3687 - val_loss: 1.4981 - val_accuracy: 0.4909
Epoch 3/5
16/18 [=========================>....] - ETA: 2:15 - loss: 1.3863 - accuracy: 0.5756

In [ ]:
model.evaluate(X_test, Y_test, verbose=1)

In [ ]:
# graph representation of accuracy and loss of train and test data 

%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel("No. of epochs")
plt.ylabel("percentage")
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'y', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel("No. of epochs")
plt.ylabel("percentage")
plt.legend()

plt.show()

In [ ]:
sentences = [
             chimp_texts[0],
             chimp_texts[1],
             chimp_texts[2],
             chimp_texts[3]
]

for sentence in sentences:
  print(sentence)
  sentence = tokenizer.texts_to_sequences([sentence])
  sentence = pad_sequences(sentence,maxlen=256, truncating='pre')
  result = np.argmax(model.predict(sentence),axis=-1)[0]
  proba = np.max(model.predict(sentence))
  print(f"{result}: {proba}\n\n")